### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
nyt_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
tmdb_url = "https://api.themoviedb.org/3/search/movie?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
# Use a sort filter, sort by newest
# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
nyt_field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Refine your search - note that these are specific to NY Times

# Filters - Technology and Special Report
nyt_filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Sort - newest
sort = "newest"

# Page number
page_number = 0

# Build NYT URL
# Build URL - The f-string has been split in multiple lines to make is easier to understand
query_url = (
    f'{nyt_url}' +
    f'api-key={nyt_api_key}' +
    f'&begin_date={begin_date}' +
    f'&end_date={end_date}' +
    f'&fq={nyt_filter_query}' + 
    f'&sort={sort}' +
    f'&fl={nyt_field_list}' +
    f'&page={str(page_number)}'
)
query_url


'https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=FKHaGic29nkCal3OLpJo5i0GchhbQLm8&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&page=0'

In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page in range(20):
    # Create query with a page number
    #API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    nyt_response = requests.get(query_url).json()

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        # Loop through the reviews["response"]["docs"] and append each review to the list
        for review in nyt_response["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Retrieved page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No results found on page {page}")
        break

Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list[:5], indent=4))

In [ ]:
# Convert the list of reviews to a pandas dataframe using normalize
#nyt_df = pd.DataFrame(reviews_list)
nyt_df = pd.json_normalize(reviews_list)
nyt_df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
nyt_df['title'] = nyt_df['headline.main'].str.extract(r'\u2018(.*?)\u2019 Review')
nyt_df

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
nyt_df['keywords'] = nyt_df['keywords'].apply(extract_keywords)


In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

titles = nyt_df['title'].to_list()

# Print the title list
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results


# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
